# set streaming application

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests

In [2]:
# create spark configuration
conf = SparkConf()
conf.setAppName("TwitterStreamApp")
# create spark context with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
# create the Streaming Context from the above spark context with interval size 2 seconds
ssc = StreamingContext(sc, 2)
# setting a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_TwitterApp")
# read data from port 9009
dataStream = ssc.socketTextStream("localhost",9009)

In [3]:
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

In [4]:
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']
def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        # create a DF from the Row RDD
        hashtags_df = sql_context.createDataFrame(row_rdd)
        # Register the dataframe as table
        hashtags_df.registerTempTable("hashtags")
        # get the top 10 hashtags from the table using SQL and print them
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
        hashtag_counts_df.show()
        # call this method to prepare top 10 hashtags DF and send them
        send_df_to_dashboard(hashtag_counts_df)
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)

In [5]:
def send_df_to_dashboard(df):
    # extract the hashtags from dataframe and convert them into array
    top_tags = [str(t.hashtag) for t in df.select("hashtag").collect()]
    # extract the counts from dataframe and convert them into array
    tags_count = [p.hashtag_count for p in df.select("hashtag_count").collect()]
    # initialize and send the data through REST API
    url = 'http://localhost:5001/updateData'
    request_data = {'label': str(top_tags), 'data': str(tags_count)}
    response = requests.post(url, data=request_data)

In [ ]:
# split each tweet into words
words = dataStream.flatMap(lambda line: line.split(" "))
# filter the words to get only hashtags, then map each hashtag to be a pair of (hashtag,1)
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))
# adding the count of each hashtag to its last count
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
# do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)
# start the streaming computation
ssc.start()
# wait for the streaming to finish
ssc.awaitTermination()

----------- 2021-02-19 17:30:52 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:30:54 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:30:56 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:30:58 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:31:00 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:31:02 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:31:04 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:31:06 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:31:08 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:31:10 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:31:12 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:31:14 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:31:16 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:31:18 -----------
Error: <class 'Value

Error: <class 'ValueError'>
----------- 2021-02-19 17:34:42 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:34:44 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:34:46 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:34:48 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:34:50 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:34:52 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:34:54 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:34:56 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:34:58 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:35:00 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:35:02 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:35:04 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:35:06 -----------
Error: <class 'ValueError'>
----------- 2021-02-19 17:35:08 ----

In [ ]:
words.first()